In [12]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pandas as pd
import importlib
import sys
import yfinance as yf

# Determine root directory - go up from notebooks folder to portfolio_analyser
if Path.cwd().name == 'notebooks':
    ROOT = Path.cwd().parent
else:
    ROOT = Path.cwd()

src_path = ROOT / 'src'

# Add src to path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import analysis and reporting functions
import technical_analysis as ta
import portfolio_reports as pr
from portfolio_reports import create_trading_playbook_pdf, create_portfolio_tracker_excel, cleanup_old_reports

# Reload modules to pick up any code changes (uncomment during development)
importlib.reload(ta)
importlib.reload(pr)

# Setup
RESULTS_DIR = ROOT / 'portfolio_results'
RESULTS_DIR.mkdir(exist_ok=True)
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

In [13]:
# ===================================================================================================
# CELL 2: LOAD PORTFOLIO DATA
# ===================================================================================================

# Load holdings and targets from data folder
holdings_file = ROOT / 'data' / 'holdings.csv'
targets_file = ROOT / 'data' / 'targets.csv'
stocks_file = ROOT / 'data' / 'stocks.txt'

# Read holdings
if not holdings_file.exists():
    print(f"Creating new holdings file: {holdings_file}")
    holdings_df = pd.DataFrame(columns=['ticker', 'quantity', 'avg_cost'])
    holdings_df.to_csv(holdings_file, index=False)
else:
    holdings_df = pd.read_csv(holdings_file)

# Read targets
if not targets_file.exists():
    print(f"Error: {targets_file} not found")
    raise FileNotFoundError(f"targets.csv required")

targets_df = pd.read_csv(targets_file)

# Convert target_pct from 0-100 to 0-1 if needed
if targets_df['target_pct'].max() > 1.0:
    targets_df['target_pct'] = targets_df['target_pct'] / 100.0

# Get list of individual tickers (no baskets)
baskets = {
    'BTC': ['BTC-USD']
}

individual_tickers = []
for ticker in targets_df['ticker']:
    if ticker in baskets:
        individual_tickers.extend(baskets[ticker])
    else:
        individual_tickers.append(ticker)

print(f"✓ Loaded {len(holdings_df)} holdings")
print(f"✓ Loaded {len(targets_df)} target allocations")
print(f"✓ Expanded to {len(individual_tickers)} individual tickers")
print(f"\nHoldings:\n{holdings_df.to_string()}")

✓ Loaded 9 holdings
✓ Loaded 7 target allocations
✓ Expanded to 7 individual tickers

Holdings:
    ticker  quantity  avg_cost last_updated  min_quantity
0     TSLA      72.0     310.6   2026-01-04           0.0
1     NVDA       0.0       0.0          NaN           0.0
2     ASML       0.0       0.0          NaN           0.0
3      AMD       0.0       0.0          NaN           0.0
4     AVGO       0.0       0.0          NaN           0.0
5     ALAB       0.0       0.0          NaN           0.0
6     MRVL       0.0       0.0          NaN           0.0
7  BTC-USD       0.5   50090.1   2026-01-04           0.5
8  SOL-USD       0.0       0.0          NaN           0.0


## Data Sanity Checks

Validate data quality before analysis:
- **Price > $0**: Catch API errors or delisted stocks
- **Fresh data**: Ensure quotes are from today (not stale cache)
- **Reasonable moves**: Flag >20% overnight changes (possible data errors)
- **Valid responses**: Confirm yfinance returned actual data

In [14]:
from datetime import datetime, timedelta

print("Running data sanity checks...")
print("=" * 80)

errors = []
warnings = []

# Check each ticker in holdings and targets
check_tickers = list(set(individual_tickers))

for ticker in check_tickers:
    try:
        # Fetch current data
        stock = yf.Ticker(ticker)
        info = stock.info
        hist = stock.history(period='5d')
        
        if hist.empty:
            errors.append(f"{ticker}: No price data returned from API")
            continue
        
        current_price = hist['Close'].iloc[-1]
        
        # Check 1: Price > $0
        if current_price <= 0:
            errors.append(f"{ticker}: Invalid price ${current_price:.2f}")
        
        # Check 2: Data freshness (updated today or yesterday for after-hours)
        last_date = hist.index[-1].date()
        today = datetime.now().date()
        yesterday = today - timedelta(days=1)
        # Allow Friday data on weekends
        friday = today - timedelta(days=2) if today.weekday() == 6 else None  # Sunday
        thursday = today - timedelta(days=3) if today.weekday() == 6 else None  # Sunday, holiday Monday
        
        if last_date < yesterday and last_date != friday and last_date != thursday:
            days_old = (today - last_date).days
            warnings.append(f"{ticker}: Data is {days_old} days old (last: {last_date})")
        
        # Check 3: Overnight move <20% (if we have previous close)
        if len(hist) >= 2:
            prev_close = hist['Close'].iloc[-2]
            pct_change = abs((current_price - prev_close) / prev_close) * 100
            if pct_change > 20:
                warnings.append(f"{ticker}: Large overnight move {pct_change:.1f}% (possible data error)")
        
        # Check 4: Valid API response (has required fields)
        if 'currentPrice' not in info and 'regularMarketPrice' not in info:
            if hist.empty:
                warnings.append(f"{ticker}: API returned limited data (using historical only)")
        
    except Exception as e:
        errors.append(f"{ticker}: API error - {str(e)}")

print(f"\n[RESULTS] Checked {len(check_tickers)} tickers\n")

if errors:
    print(f"[ERROR] {len(errors)} CRITICAL ISSUES:")
    for err in errors:
        print(f"  {err}")
    print("\n[ACTION] Fix these issues before trading!")
else:
    print("[OK] No critical errors found")

if warnings:
    print(f"\n[WARN] {len(warnings)} warnings:")
    for warn in warnings:
        print(f"  {warn}")
else:
    print("[OK] No warnings")

print("\n" + "=" * 80)

Running data sanity checks...

[RESULTS] Checked 7 tickers

[OK] No critical errors found
[OK] No warnings



In [15]:
# Set actual cash available
CASH_AVAILABLE = 110439.86

# Calculate initial portfolio total from holdings at avg_cost
PORTFOLIO_TOTAL = 0
initial_holdings_value = 0

for _, row in holdings_df.iterrows():
    quantity = row['quantity']
    if quantity > 0:
        initial_holdings_value += quantity * row['avg_cost']

PORTFOLIO_TOTAL = initial_holdings_value

print(f"Cash Available: ${CASH_AVAILABLE:,.2f}")
print(f"Initial Holdings Value (avg cost): ${initial_holdings_value:,.2f}")
print(f"Initial Portfolio Total (avg cost): ${PORTFOLIO_TOTAL:,.2f}")
print(f"Total Portfolio Value (avg cost + cash): ${PORTFOLIO_TOTAL + CASH_AVAILABLE:,.2f}")
print(f"\nNote: Target values will be calculated after fetching current market prices")

Cash Available: $110,439.86
Initial Holdings Value (avg cost): $47,408.25
Initial Portfolio Total (avg cost): $47,408.25
Total Portfolio Value (avg cost + cash): $157,848.11

Note: Target values will be calculated after fetching current market prices


In [16]:
# ===================================================================================================
# CELL 3: ANALYZE PORTFOLIO STOCKS
# ===================================================================================================

print("Starting portfolio analysis...")
print(f"Stocks to analyze: {len(individual_tickers)}")
print(f"Concurrency: 2 threads")
print(f"Cache: 24hr TTL, 1-2s delay between API calls\n")

results = []
errors = []

def analyze_stock(ticker):
    """Analyze single stock with error handling"""
    try:
        result = ta.analyze_ticker(ticker)
        if result:
            return ticker, result, None
    except Exception as e:
        return ticker, None, str(e)
    return ticker, None, "Unknown error"

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = {executor.submit(analyze_stock, ticker): ticker for ticker in individual_tickers}

    for future in as_completed(futures):
        ticker, result, error = future.result()

        if result and 'error' not in result:
            results.append(result)
            print(f"✓ {ticker}: {result['signal']}")
        else:
            error_msg = result.get('error', error) if result else error
            errors.append((ticker, error_msg))
            print(f"✗ {ticker}: {error_msg}")

print(f"\n{'='*80}")

print(f"Analysis complete: {len(results)} success, {len(errors)} errors")
print(f"{'='*80}")

Starting portfolio analysis...
Stocks to analyze: 7
Concurrency: 2 threads
Cache: 24hr TTL, 1-2s delay between API calls

✓ TSLA: FULL HOLD + ADD
✓ NVDA: FULL HOLD + ADD
✓ ASML: FULL HOLD + ADD
✓ AVGO: HOLD
✓ AMD: HOLD
✓ ALAB: HOLD MOST + REDUCE
✓ MRVL: FULL HOLD + ADD

Analysis complete: 7 success, 0 errors


In [17]:
# Recalculate PORTFOLIO_TOTAL using current prices from results
PORTFOLIO_TOTAL_CURRENT = 0

print("\nRecalculating portfolio with current prices:")
for _, row in holdings_df.iterrows():
    ticker = row['ticker']
    quantity = row['quantity']
    if quantity > 0:
        # Find the result for this ticker
        result = next((r for r in results if r['ticker'] == ticker), None)
        if result:
            current_price = result['current_price']
            current_value = quantity * current_price
            PORTFOLIO_TOTAL_CURRENT += current_value
            print(f"{ticker}: {quantity:.4f} quantity @ ${current_price:.2f} = ${current_value:,.2f}")

PORTFOLIO_TOTAL = PORTFOLIO_TOTAL_CURRENT
total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE

print(f"\n✓ Updated PORTFOLIO_TOTAL (current prices) = ${PORTFOLIO_TOTAL:,.2f}")
print(f"✓ Total Portfolio Value (current + cash) = ${total_portfolio_value:,.2f}")

# NOW calculate target_value based on CURRENT portfolio value
print(f"\nAuto-calculating target values based on CURRENT portfolio value ${total_portfolio_value:,.2f}...")
targets_df['target_value'] = (targets_df['target_pct'] * total_portfolio_value).round(0).astype(int)

# Save updated targets back to CSV
targets_df.to_csv(targets_file, index=False)
print(f"✓ Updated target values in {targets_file.name}")

# Display updated targets
print(f"\nUpdated Target Allocations (based on current prices):")
for _, row in targets_df.iterrows():
    print(f"  {row['ticker']:<10} {row['target_pct']*100:>3.0f}% → ${row['target_value']:>10,}")

total_target_value = targets_df['target_value'].sum()
print(f"\nTotal Target Value: ${total_target_value:,.2f} ({total_target_value/total_portfolio_value*100:.1f}%)")


Recalculating portfolio with current prices:
TSLA: 72.0000 quantity @ $450.09 = $32,406.48

✓ Updated PORTFOLIO_TOTAL (current prices) = $32,406.48
✓ Total Portfolio Value (current + cash) = $142,846.34

Auto-calculating target values based on CURRENT portfolio value $142,846.34...
✓ Updated target values in targets.csv

Updated Target Allocations (based on current prices):
  TSLA        50% → $    71,423
  NVDA        10% → $    14,285
  ASML        10% → $    14,285
  AMD         10% → $    14,285
  AVGO        10% → $    14,285
  ALAB         5% → $     7,142
  MRVL         5% → $     7,142

Total Target Value: $142,847.00 (100.0%)


In [18]:
# ===================================================================================================
# CELL 5: BUILD PORTFOLIO POSITIONS
# ===================================================================================================

portfolio_positions = []
buy_count = 0
sell_count = 0
hold_count = 0

for result in results:
    ticker = result['ticker']

    # Find target allocation
    target = targets_df[targets_df['ticker'] == ticker]
    if target.empty:
        print(f"Warning: No target allocation for {ticker}")
        continue

    target_pct = target.iloc[0]['target_pct']

    # Find holding
    holding = holdings_df[holdings_df['ticker'] == ticker]
    quantity = holding.iloc[0]['quantity'] if not holding.empty else 0

    # Calculate current value and position metrics
    current_price = result['current_price']
    current_value = quantity * current_price

    # Calculate min quantity (not tradeable below this)
    min_quantity = holding.iloc[0].get('min_quantity', 0) if not holding.empty else 0
    min_value = min_quantity * current_price
    tradeable_quantity = max(0, quantity - min_quantity)
    tradeable_value = tradeable_quantity * current_price

    # Calculate position gap
    total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE
    current_pct = (current_value / total_portfolio_value) if total_portfolio_value > 0 else 0
    gap_value = (target_pct * total_portfolio_value) - current_value

    # Position gap structure for MA feasibility checks
    position_gap = {
        'gap_value': gap_value,
        'target_pct': target_pct,
        'current_pct': current_pct
    }

    # Check if MAs block path to resistance
    adjusted_r1, adjusted_r2, adjusted_r3, ma_note = ta.adjust_sell_levels_for_mas(
        d50=result.get('d50'),
        d100=result.get('d100'),
        d200=result.get('d200'),
        r1=result.get('r1'),
        r2=result.get('r2'),
        r3=result.get('r3'),
        current_price=current_price
    )

    # Determine portfolio action
    action = ta.determine_portfolio_action(
        signal=result['signal'],
        position_gap=position_gap,
        buy_quality=result.get('buy_quality', 'N/A')
    )

    # Count actions
    if action == "BUY":
        buy_count += 1
    elif action == "SELL":
        sell_count += 1
    else:
        hold_count += 1

    # Calculate buy tranches with individual quality ratings
    buy_tranches = ta.calculate_buy_tranches(
        gap_value=gap_value,
        s1=result.get('s1'),
        s2=result.get('s2'),
        s3=result.get('s3'),
        current_price=result['current_price'],
        s1_quality=(result.get('s1_quality', 'N/A'), result.get('s1_quality_note', '')),
        s2_quality=(result.get('s2_quality', 'N/A'), result.get('s2_quality_note', '')),
        s3_quality=(result.get('s3_quality', 'N/A'), result.get('s3_quality_note', ''))
    )

    # Calculate sell tranches with individual quality ratings
    sell_tranches = ta.calculate_sell_tranches(
        current_value=tradeable_value,
        signal=result['signal'],
        r1=result.get('r1'),
        r2=result.get('r2'),
        r3=result.get('r3'),
        adjusted_r1=adjusted_r1,
        adjusted_r2=adjusted_r2,
        adjusted_r3=adjusted_r3,
        r1_quality=(result.get('r1_quality', 'N/A'), result.get('r1_quality_note', '')),
        r2_quality=(result.get('r2_quality', 'N/A'), result.get('r2_quality_note', '')),
        r3_quality=(result.get('r3_quality', 'N/A'), result.get('r3_quality_note', ''))
    )

    # Build position dict
    position = {
        'ticker': ticker,
        'signal': result['signal'],
        'current_price': result['current_price'],
        'quantity': quantity,
        'min_quantity': min_quantity,
        'tradeable_quantity': tradeable_quantity,
        'current_value': current_value,
        'tradeable_value': tradeable_value,
        'min_value': min_value,
        'target_pct': target_pct,
        'current_pct': current_pct,
        'gap_value': gap_value,
        'action': action,
        'buy_quality': result.get('buy_quality', 'N/A'),
        'buy_quality_note': result.get('buy_quality_note', ''),
        's1_quality': result.get('s1_quality', 'N/A'),
        's1_quality_note': result.get('s1_quality_note', ''),
        's2_quality': result.get('s2_quality', 'N/A'),
        's2_quality_note': result.get('s2_quality_note', ''),
        's3_quality': result.get('s3_quality', 'N/A'),
        's3_quality_note': result.get('s3_quality_note', ''),
        'r1_quality': result.get('r1_quality', 'N/A'),
        'r1_quality_note': result.get('r1_quality_note', ''),
        'r2_quality': result.get('r2_quality', 'N/A'),
        'r2_quality_note': result.get('r2_quality_note', ''),
        'r3_quality': result.get('r3_quality', 'N/A'),
        'r3_quality_note': result.get('r3_quality_note', ''),
        'buy_tranches': buy_tranches,
        'sell_tranches': sell_tranches,
        'sell_feasibility_note': ma_note,
        # Technical levels for Technical Levels tab
        'rsi': result.get('rsi'),
        'bb_upper': result.get('bb_upper'),
        'bb_middle': result.get('bb_middle'),
        'bb_lower': result.get('bb_lower'),
        'd50': result.get('d50'),
        'd100': result.get('d100'),
        'd200': result.get('d200'),
        's1': result.get('s1'),
        's2': result.get('s2'),
        's3': result.get('s3'),
        'r1': result.get('r1'),
        'r2': result.get('r2'),
        'r3': result.get('r3'),
        # Volume Profile (VRVP) data
        'poc_60d': result.get('poc_60d'),
        'vah_60d': result.get('vah_60d'),
        'val_60d': result.get('val_60d'),
        'hvn_above_60d': result.get('hvn_above_60d'),
        'hvn_below_60d': result.get('hvn_below_60d'),
        'lvn_above_60d': result.get('lvn_above_60d'),
        'lvn_below_60d': result.get('lvn_below_60d'),
        'poc_52w': result.get('poc_52w'),
        'vah_52w': result.get('vah_52w'),
        'val_52w': result.get('val_52w')
    }

    portfolio_positions.append(position)

total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE
cash_pct_corrected = (CASH_AVAILABLE / total_portfolio_value) * 100

# Build portfolio_data structure expected by reporting functions
portfolio_data = {
    'positions': portfolio_positions,
    'portfolio_total': PORTFOLIO_TOTAL,
    'summary': {
        'buy_count': buy_count,
        'sell_count': sell_count,
        'hold_count': hold_count,
        'total_portfolio_value': total_portfolio_value,
        'cash_available': CASH_AVAILABLE
    }
}

print(f"\nPortfolio Summary:")
print(f"  Total Value: ${total_portfolio_value:,.2f}")
print(f"  Current Holdings: ${PORTFOLIO_TOTAL:,.2f}")
print(f"  Cash Available: ${CASH_AVAILABLE:,.2f} ({cash_pct_corrected:.1f}%)")
print(f"\nAction Summary:")
print(f"  BUY signals: {buy_count}")
print(f"  SELL signals: {sell_count}")
print(f"  HOLD signals: {hold_count}")



Portfolio Summary:
  Total Value: $142,846.34
  Current Holdings: $32,406.48
  Cash Available: $110,439.86 (77.3%)

Action Summary:
  BUY signals: 1
  SELL signals: 0
  HOLD signals: 6


In [19]:
print("\n" + "="*80)
print("🔄 SIGNAL CHANGES - What's Different Today")
print("="*80)

# Look for yesterday's tracker file to compare (exclude files from today)
from datetime import datetime
today_prefix = f"portfolio_tracker_{datetime.now().strftime('%Y%m%d')}"
all_files = sorted(RESULTS_DIR.glob('portfolio_tracker_*.xlsx'), key=lambda p: p.stem, reverse=True)
yesterday_files = [f for f in all_files if not f.name.startswith(today_prefix)]

if len(yesterday_files) >= 1:
    # Get the most recent file from a previous date
    yesterday_file = yesterday_files[0]
    
    try:
        # Read yesterday's signals from Action Plan sheet
        try:
            yesterday_df = pd.read_excel(yesterday_file, sheet_name='Action Plan', skiprows=5)
        except ValueError:
            # Action Plan sheet doesn't exist, skip comparison
            print(f"\n⚠️  Yesterday's file doesn't have 'Action Plan' sheet")
            print(f"   Skipping comparison - run notebook again tomorrow")
            yesterday_df = None
        
        if yesterday_df is not None:
            # Normalize column names to handle different formats
            yesterday_df.columns = yesterday_df.columns.str.strip().str.title()
            
            # Check if required columns exist
            if 'Ticker' not in yesterday_df.columns or 'Signal' not in yesterday_df.columns:
                print(f"\n⚠️  Yesterday's file has unexpected format")
                print(f"   Available columns: {', '.join(yesterday_df.columns.tolist())}")
                print(f"   Expected: 'Ticker', 'Signal'")
                print(f"   Skipping comparison - run notebook again tomorrow")
            else:
                # Create today's signal dictionary
                today_signals = {pos['ticker']: pos['signal'] for pos in portfolio_positions}
                
                # Get unique ticker-signal pairs from yesterday (Action Plan may have duplicate tickers)
                yesterday_signals_df = yesterday_df[['Ticker', 'Signal']].drop_duplicates(subset=['Ticker'])
                
                # Track changes
                signal_changes = []
                new_buys = []
                new_sells = []
                
                for ticker, today_signal in today_signals.items():
                    yesterday_row = yesterday_signals_df[yesterday_signals_df['Ticker'] == ticker]
                    
                    if yesterday_row.empty:
                        # New ticker
                        if today_signal == "FULL HOLD + ADD":
                            new_buys.append((ticker, "NEW", today_signal))
                    else:
                        yesterday_signal = yesterday_row.iloc[0]['Signal']
                        
                        if yesterday_signal != today_signal:
                            signal_changes.append((ticker, yesterday_signal, today_signal))
                            
                            # Check if it's a new buy/sell opportunity
                            if yesterday_signal != "FULL HOLD + ADD" and today_signal == "FULL HOLD + ADD":
                                new_buys.append((ticker, yesterday_signal, today_signal))
                            elif yesterday_signal not in ["HOLD MOST + REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"] and today_signal in ["HOLD MOST + REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]:
                                new_sells.append((ticker, yesterday_signal, today_signal))
                
                # Display changes
                if signal_changes:
                    print(f"\n🔔 SIGNAL CHANGES ({len(signal_changes)} stocks)")
                    for ticker, old_signal, new_signal in signal_changes:
                        # Determine emoji
                        if new_signal == "FULL HOLD + ADD":
                            emoji = "🟢⬆️"  # Improved to buy
                        elif new_signal in ["HOLD MOST + REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]:
                            emoji = "🔴⬇️"  # Weakened to defensive
                        elif old_signal in ["HOLD MOST + REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]:
                            emoji = "🟢⬆️"  # Improved from defensive
                        else:
                            emoji = "⚪➡️"  # Neutral change
                        
                        print(f"   {emoji} {ticker:<10} {old_signal:<20} → {new_signal}")
                else:
                    print(f"\n✅ NO SIGNAL CHANGES")
                    print(f"   All signals unchanged from yesterday")
                
                # Highlight new buy opportunities
                if new_buys:
                    print(f"\n🟢 NEW BUY OPPORTUNITIES ({len(new_buys)})")
                    for ticker, old_signal, new_signal in new_buys:
                        pos = next((p for p in portfolio_positions if p['ticker'] == ticker), None)
                        if pos:
                            buy_quality = pos['buy_quality']
                            s1 = pos.get('s1')
                            s1_str = f"S1: ${s1:.2f}" if s1 else "No S1"
                            print(f"   💚 {ticker:<10} {old_signal:<20} → {new_signal:<20} | {buy_quality:<12} | {s1_str}")
                
                # Highlight new defensive actions
                if new_sells:
                    print(f"\n🔴 NEW DEFENSIVE SIGNALS ({len(new_sells)})")
                    for ticker, old_signal, new_signal in new_sells:
                        pos = next((p for p in portfolio_positions if p['ticker'] == ticker), None)
                        if pos:
                            tradeable_value = pos['tradeable_value']
                            print(f"   ⚠️  {ticker:<10} {old_signal:<20} → {new_signal:<20} | Position: ${tradeable_value:>10,.0f}")
                
                # Price changes for active positions
                print(f"\n📊 PRICE CHANGES (Active Positions)")
                for pos in portfolio_positions:
                    if pos['quantity'] > 0:
                        ticker = pos['ticker']
                        today_price = pos['current_price']
                        
                        yesterday_row = yesterday_signals_df[yesterday_signals_df['Ticker'] == ticker]
                        if not yesterday_row.empty:
                            # Try to get price from Action Plan sheet
                            yesterday_price_rows = yesterday_df[yesterday_df['Ticker'] == ticker]
                            if not yesterday_price_rows.empty and 'Price' in yesterday_price_rows.columns:
                                price_str = str(yesterday_price_rows.iloc[0].get('Price', '')).replace('$', '').replace(',', '')
                                try:
                                    yesterday_price = float(price_str) if price_str and price_str != '-' else today_price
                                except:
                                    yesterday_price = today_price
                            else:
                                yesterday_price = today_price
                            
                            price_change = today_price - yesterday_price
                            price_change_pct = (price_change / yesterday_price * 100) if yesterday_price > 0 else 0
                            
                            if abs(price_change_pct) >= 1.0:  # Only show if changed by 1% or more
                                emoji = "📈" if price_change > 0 else "📉" if price_change < 0 else "➡️"
                                print(f"   {emoji} {ticker:<10} ${yesterday_price:>7,.2f} → ${today_price:>7,.2f} ({price_change_pct:>+6.2f}%)")
                
                print(f"\n✅ Compared to: {yesterday_file.name} (from previous date)")
        
    except Exception as e:
        print(f"\n⚠️  Could not compare to yesterday's file: {e}")
        print(f"   Run this notebook tomorrow to see changes!")

else:
    print(f"\n⚠️  NO PREVIOUS DATA TO COMPARE")
    print(f"   This is your first run or no files from previous dates exist")
    print(f"   Run this notebook again tomorrow to see changes!")

print("\n" + "="*80)


🔄 SIGNAL CHANGES - What's Different Today

⚠️  NO PREVIOUS DATA TO COMPARE
   This is your first run or no files from previous dates exist
   Run this notebook again tomorrow to see changes!



## 🔄 WHAT CHANGED TODAY

Compare today's signals vs yesterday to focus on NEW developments.

In [20]:
print("\n" + "="*80)
print("📊 PORTFOLIO DASHBOARD - QUICK VIEW")
print("="*80)

# Portfolio Summary
total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE
cash_pct = (CASH_AVAILABLE / total_portfolio_value) * 100 if total_portfolio_value > 0 else 0

print(f"\n💰 PORTFOLIO VALUE: ${total_portfolio_value:,.2f}")
print(f"   Holdings: ${PORTFOLIO_TOTAL:,.2f}")
print(f"   Cash: ${CASH_AVAILABLE:,.2f} ({cash_pct:.1f}%)")

# Active Positions (holdings with quantity > 0)
active_positions = [p for p in portfolio_positions if p['quantity'] > 0]
if active_positions:
    print(f"\n📈 ACTIVE POSITIONS ({len(active_positions)})")
    for pos in active_positions:
        ticker = pos['ticker']
        signal = pos['signal']
        quantity = pos['quantity']
        current_price = pos['current_price']
        current_value = pos['current_value']
        
        # Calculate P&L if we have holdings data
        holding = holdings_df[holdings_df['ticker'] == ticker]
        if not holding.empty and quantity > 0:
            avg_cost = holding.iloc[0]['avg_cost']
            total_cost = quantity * avg_cost
            pnl = current_value - total_cost
            pnl_pct = (pnl / total_cost * 100) if total_cost > 0 else 0
            
            # Signal emoji
            if signal == "FULL HOLD + ADD":
                emoji = "✅"
            elif signal in ["HOLD MOST + REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]:
                emoji = "🔴"
            else:
                emoji = "⚪"
            
            print(f"   {emoji} {ticker:<10} {signal:<20} | ${current_value:>10,.0f} | P&L: {pnl:>+10,.0f} ({pnl_pct:>+6.1f}%)")

# Watchlist - Buy Opportunities (no current holdings but buy signal)
watchlist_buys = [p for p in portfolio_positions if p['quantity'] == 0 and p['signal'] == "FULL HOLD + ADD" and p['buy_quality'] in ['EXCELLENT', 'GOOD', 'OK']]
if watchlist_buys:
    print(f"\n🔍 WATCHLIST - BUY OPPORTUNITIES ({len(watchlist_buys)})")
    for pos in watchlist_buys:
        ticker = pos['ticker']
        buy_quality = pos['buy_quality']
        s1 = pos.get('s1')
        current_price = pos['current_price']
        gap = pos['gap_value']
        
        s1_str = f"S1: ${s1:.2f}" if s1 else "No S1"
        print(f"   🟢 {ticker:<10} {buy_quality:<12} | Buy {s1_str:<15} | Current: ${current_price:>7,.2f} | Need: ${gap:>10,.0f}")

# Defensive Actions (bearish signals)
defensive_actions = [p for p in portfolio_positions if p['quantity'] > 0 and p['signal'] in ["HOLD MOST + REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]]
if defensive_actions:
    print(f"\n⚠️  DEFENSIVE ACTIONS ({len(defensive_actions)})")
    for pos in defensive_actions:
        ticker = pos['ticker']
        signal = pos['signal']
        tradeable_value = pos['tradeable_value']
        
        # Calculate reduction amount
        if signal == "HOLD MOST + REDUCE":
            reduce_pct = 20
        elif signal == "REDUCE":
            reduce_pct = 40
        elif signal == "LIGHT / CASH":
            reduce_pct = 60
        elif signal == "CASH":
            reduce_pct = 80
        else:  # FULL CASH / DEFEND
            reduce_pct = 100
        
        reduce_amount = tradeable_value * (reduce_pct / 100)
        print(f"   🔴 {ticker:<10} {signal:<20} | Reduce {reduce_pct}% (~${reduce_amount:>10,.0f})")
else:
    print(f"\n⚠️  DEFENSIVE ACTIONS (0)")
    print(f"   ✅ No bearish signals - all positions clear")

# Action Summary
print(f"\n📋 TODAY'S ACTIONS")
print(f"   {buy_count} BUY opportunities")
print(f"   {sell_count} SELL/REDUCE actions")
print(f"   {hold_count} HOLD (no action)")

print("\n" + "="*80)


📊 PORTFOLIO DASHBOARD - QUICK VIEW

💰 PORTFOLIO VALUE: $142,846.34
   Holdings: $32,406.48
   Cash: $110,439.86 (77.3%)

📈 ACTIVE POSITIONS (1)
   ✅ TSLA       FULL HOLD + ADD      | $    32,406 | P&L:    +10,043 ( +44.9%)

🔍 WATCHLIST - BUY OPPORTUNITIES (1)
   🟢 MRVL       GOOD         | Buy S1: $81.12      | Current: $  83.12 | Need: $     7,142

⚠️  DEFENSIVE ACTIONS (0)
   ✅ No bearish signals - all positions clear

📋 TODAY'S ACTIONS
   1 BUY opportunities
   0 SELL/REDUCE actions
   6 HOLD (no action)



## 📊 5-SECOND DASHBOARD

Quick visual summary of your portfolio - see everything at a glance before diving into details.

In [21]:
# ===================================================================================================
# CELL 6: GENERATE REPORTS
# ===================================================================================================

# Generate Portfolio Playbook PDF
print("\n=== Generating Portfolio Playbook PDF ===")
pdf_path = RESULTS_DIR / f'portfolio_playbook_{TIMESTAMP}.pdf'
create_trading_playbook_pdf(portfolio_data, pdf_path, TIMESTAMP)
print(f"✓ Created: {pdf_path.name}")

# Generate Portfolio Tracker Excel
print("\n=== Generating Portfolio Tracker Excel ===")
excel_path = RESULTS_DIR / f'portfolio_tracker_{TIMESTAMP}.xlsx'
create_portfolio_tracker_excel(portfolio_data, excel_path)
print(f"✓ Created: {excel_path.name}")

# Archive old reports
print("\n=== Cleaning Up Old Reports ===")
cleanup_old_reports(RESULTS_DIR, max_files=1)

print(f"\n{'='*80}")
print("✅ PORTFOLIO ANALYSIS COMPLETE")
print(f"{'='*80}")
print(f"\nGenerated Files:")
print(f"  1. {pdf_path.name}")
print(f"  2. {excel_path.name}")
print(f"\nLocation: {RESULTS_DIR.absolute()}")



=== Generating Portfolio Playbook PDF ===
✓ Created: portfolio_playbook_20260112_202125.pdf

=== Generating Portfolio Tracker Excel ===
✓ Created: portfolio_tracker_20260112_202125.xlsx

=== Cleaning Up Old Reports ===
  📦 Archived: portfolio_playbook_20260112_185656.pdf
  📦 Archived: portfolio_tracker_20260112_185656.xlsx
  ✅ Archived 2 file(s), kept 1 most recent

✅ PORTFOLIO ANALYSIS COMPLETE

Generated Files:
  1. portfolio_playbook_20260112_202125.pdf
  2. portfolio_tracker_20260112_202125.xlsx

Location: c:\workspace\portfolio_analyser\portfolio_results


## Signal Performance Tracker

Track signal outcomes over time and view win rates. Automatically logs all "FULL HOLD + ADD" signals and calculates returns 30/60/90 days later to build confidence in the system.

In [22]:
from signal_tracker import log_signal, update_returns, get_signal_performance

print("=" * 80)
print("SIGNAL PERFORMANCE TRACKER")
print("=" * 80)

history_file = ROOT / 'data' / 'signal_history.csv'

# Step 1: Log any new FULL HOLD + ADD signals from today's analysis
logged_count = 0
for result in results:
    if result['signal'] == 'FULL HOLD + ADD':
        log_signal(
            ticker=result['ticker'],
            signal=result['signal'],
            price=result['current_price'],
            buy_quality=result.get('buy_quality', ''),
            history_file=history_file
        )
        logged_count += 1

if logged_count > 0:
    print(f"\n[>>] Logged {logged_count} new signals")
else:
    print(f"\n[OK] No new signals to log")

# Step 2: Update returns for past signals
print("\n[INFO] Updating historical returns (30/60/90 day)...")
update_returns(history_file=history_file, lookback_days=120)

# Step 3: Display performance statistics
print("\n" + "=" * 80)
print("PERFORMANCE SUMMARY - FULL HOLD + ADD Signals (EXCELLENT/GOOD/OK)")
print("=" * 80)

stats = get_signal_performance(
    signal_type='FULL HOLD + ADD',
    history_file=history_file,
    min_quality='OK'  # Only EXCELLENT, GOOD, OK (exclude EXTENDED, WEAK)
)

if stats['total_signals'] > 0:
    print(f"\nTotal signals tracked: {stats['total_signals']}")
    
    print(f"\n30-DAY PERFORMANCE:")
    if stats['signals_with_data_30d'] > 0:
        print(f"  Win rate: {stats['win_rate_30d']:.1f}% ({int(stats['win_rate_30d'] * stats['signals_with_data_30d'] / 100)}/{stats['signals_with_data_30d']} wins)")
        print(f"  Avg return: {stats['avg_return_30d']:+.1f}%")
        print(f"  Median: {stats['median_return_30d']:+.1f}%")
        print(f"  Best: {stats['best_return_30d']:+.1f}%")
        print(f"  Worst: {stats['worst_return_30d']:+.1f}%")
    else:
        print(f"  [WAIT] Need 30 days for first results")
    
    print(f"\n60-DAY PERFORMANCE:")
    if stats['signals_with_data_60d'] > 0:
        print(f"  Win rate: {stats['win_rate_60d']:.1f}% ({int(stats['win_rate_60d'] * stats['signals_with_data_60d'] / 100)}/{stats['signals_with_data_60d']} wins)")
        print(f"  Avg return: {stats['avg_return_60d']:+.1f}%")
        print(f"  Median: {stats['median_return_60d']:+.1f}%")
        print(f"  Best: {stats['best_return_60d']:+.1f}%")
        print(f"  Worst: {stats['worst_return_60d']:+.1f}%")
    else:
        print(f"  [WAIT] Need 60 days for first results")
    
    print(f"\n90-DAY PERFORMANCE:")
    if stats['signals_with_data_90d'] > 0:
        print(f"  Win rate: {stats['win_rate_90d']:.1f}% ({int(stats['win_rate_90d'] * stats['signals_with_data_90d'] / 100)}/{stats['signals_with_data_90d']} wins)")
        print(f"  Avg return: {stats['avg_return_90d']:+.1f}%")
        print(f"  Median: {stats['median_return_90d']:+.1f}%")
        print(f"  Best: {stats['best_return_90d']:+.1f}%")
        print(f"  Worst: {stats['worst_return_90d']:+.1f}%")
    else:
        print(f"  [WAIT] Need 90 days for first results")
    
    print(f"\n[INFO] Building historical data - full stats available after 90 days")
else:
    print("\n[INFO] No signals tracked yet")
    print("[INFO] Start tracking: signals logged automatically when found")

print("\n" + "=" * 80)

SIGNAL PERFORMANCE TRACKER

[>>] Logged 4 new signals

[INFO] Updating historical returns (30/60/90 day)...

PERFORMANCE SUMMARY - FULL HOLD + ADD Signals (EXCELLENT/GOOD/OK)

Total signals tracked: 4

30-DAY PERFORMANCE:
  [WAIT] Need 30 days for first results

60-DAY PERFORMANCE:
  [WAIT] Need 60 days for first results

90-DAY PERFORMANCE:
  [WAIT] Need 90 days for first results

[INFO] Building historical data - full stats available after 90 days

